In [1]:
import numpy as np
import pandas as pd

In [11]:
x = float('-inf')  # wall punish
r = np.matrix([
    [x, 0, x, 0],  # 1
    [x, 0, 0, -10],  # 2
    [x, 100, 0, x],  # 3
    [0, x, x, 0],  # 4
    [0, x, 0, 100],  # 5
    [-10, x, 0, x],  # 6
])

,up,down,left,right
s0,-inf,0.0,-inf,0.0
s1,-inf,0.0,0.0,-10.0
s2,-inf,100.0,0.0,-inf
s3,0.0,-inf,-inf,0.0
s4,0.0,-inf,0.0,100.0
s5,-10.0,-inf,0.0,-inf


In [39]:
def get_df(data):
    return pd.DataFrame(
        data,
        columns=["up", "down", "left", "right"],
        index=["s" + str(i) for i in range(1, data.shape[0] + 1)])

In [40]:
# let's see what the reward table r looks like
df_r = get_df(r)
df_r

,up,down,left,right
s1,-inf,0.0,-inf,0.0
s2,-inf,0.0,0.0,-10.0
s3,-inf,100.0,0.0,-inf
s4,0.0,-inf,-inf,0.0
s5,0.0,-inf,0.0,100.0
s6,-10.0,-inf,0.0,-inf


In [41]:
# and here is the initialized q-table
q_table = np.zeros((6, 4))
df_q_table = get_df(q_table)
df_q_table

,up,down,left,right
s1,0.0,0.0,0.0,0.0
s2,0.0,0.0,0.0,0.0
s3,0.0,0.0,0.0,0.0
s4,0.0,0.0,0.0,0.0
s5,0.0,0.0,0.0,0.0
s6,0.0,0.0,0.0,0.0


In [42]:
discount_factor = 0.9
learning_rate = 1

In [43]:
s1, s2, s3, s4, s5, s6 = 0, 1, 2, 3, 4, 5
up, down, left, right = 0, 1, 2, 3

In [69]:
def get_new_q(state, action, next_state, print_out=True):
    reward = r[state, action]  # reward for current state
    max_next = np.max(q_table[next_state, :])  # max for all actions of next state
    new_q = reward + discount_factor * max_next  # new q for q-table
    if print_out:
        print('reward: ', reward)
        print('max_next: ', max_next)
        print('new_q: ', new_q)
    return new_q

The robot is placed in state 4 first and consider a move to 5. Calculate the new Q-value for this state-action pair and update the Q matrix.


In [56]:
state = s4
action = right
next_state = s5

q_table[state, action] = get_new_q(state, action, next_state)
df_q_table = get_df(q_table)
df_q_table

reward:  0.0
max_next:  100.0
new_q:  90.0


,up,down,left,right
s1,0.0,81.0,0.0,0.0
s2,0.0,0.0,0.0,0.0
s3,0.0,0.0,0.0,0.0
s4,0.0,0.0,0.0,90.0
s5,0.0,0.0,0.0,100.0
s6,0.0,0.0,0.0,0.0


The robot is placed in state 5 and consider the move to 6. Calculate the new Q-value for this state-action pair and update the Q matrix.

In [57]:
state = s5
action = right
next_state = s6

q_table[state, action] = get_new_q(state, action, next_state)
df_q_table = get_df(q_table)
df_q_table

reward:  100.0
max_next:  0.0
new_q:  100.0


,up,down,left,right
s1,0.0,81.0,0.0,0.0
s2,0.0,0.0,0.0,0.0
s3,0.0,0.0,0.0,0.0
s4,0.0,0.0,0.0,90.0
s5,0.0,0.0,0.0,100.0
s6,0.0,0.0,0.0,0.0


The robot is placed in state 4 again and considers the move to 5. Calculate the new Q-value for this state-action pair and update the Q matrix.


In [58]:
state = s4
action = right
next_state = s5

q_table[state, action] = get_new_q(state, action, next_state)
df_q_table = get_df(q_table)
df_q_table

reward:  0.0
max_next:  100.0
new_q:  90.0


,up,down,left,right
s1,0.0,81.0,0.0,0.0
s2,0.0,0.0,0.0,0.0
s3,0.0,0.0,0.0,0.0
s4,0.0,0.0,0.0,90.0
s5,0.0,0.0,0.0,100.0
s6,0.0,0.0,0.0,0.0


The robot is placed in state 1 and consider the move to 4. Calculate the new Q-value for this state-action pair and update the Q matrix.

In [63]:
state = s1
action = down
next_state = s4

q_table[state, action] = get_new_q(state, action, next_state)
df_q_table = get_df(q_table)
df_q_table

reward:  0.0
max_next:  90.0
new_q:  81.0


,up,down,left,right
s1,0.0,81.0,0.0,0.0
s2,0.0,0.0,0.0,0.0
s3,0.0,0.0,0.0,0.0
s4,0.0,0.0,0.0,90.0
s5,0.0,0.0,0.0,100.0
s6,0.0,0.0,0.0,0.0


Based on the information entered, which path is the most likely choice of the robot at this stage when dropped in zone 2?

Checking new q's for different actions

In [76]:
from_s2_to = {
    "down": get_new_q(s2, down, s5, False),
    "left": get_new_q(s2, left, s1, False),
    "right": get_new_q(s2, right, s3, False),
}
df_from_s2 = pd.DataFrame.from_dict(from_s2_to, orient='index', columns=['Q'])
df_from_s2

,Q
down,90.0
left,72.9
right,-10.0


The highest q is action **down** so this is the most likely choice